In [ ]:
from immutables import Map
from pydantic import BaseModel
from typing import Type, Any, Mapping, get_origin, get_args
from typing_extensions import TypedDict
import inspect

# 動態生成 TypedDict 的輔助函數
def generate_typed_dict(model_cls: Type[BaseModel], name: str) -> Type[TypedDict]:
    annotations = {}
    for field_name, field_info in model_cls.__fields__.items():
        field_type = field_info.annotation
        # 處理嵌套的 Pydantic 模型
        if inspect.isclass(field_type) and issubclass(field_type, BaseModel):
            nested_typed_dict = generate_typed_dict(field_type, f"{name}_{field_name.capitalize()}Dict")
            annotations[field_name] = nested_typed_dict
        else:
            annotations[field_name] = field_type
    return TypedDict(name, annotations)

# 將 Pydantic 模型轉為 immutables.Map 的輔助函數
def pydantic_to_immutable_map(model: BaseModel) -> Mapping[str, Any]:
    data = model.dict()
    
    def convert_to_immutable(obj: Any) -> Any:
        if isinstance(obj, BaseModel):
            return pydantic_to_immutable_map(obj)
        elif isinstance(obj, dict):
            return Map({k: convert_to_immutable(v) for k, v in obj.items()})
        elif isinstance(obj, list):
            return [convert_to_immutable(item) for item in obj]
        else:
            return obj
    
    return convert_to_immutable(data)

# 裝飾器：為 Pydantic 模型添加 immutables.Map 轉換和型態註解
def immutable_map_model(cls: Type[BaseModel]) -> Type[BaseModel]:
    # 生成 TypedDict 型態
    typed_dict_name = f"{cls.__name__}Dict"
    typed_dict = generate_typed_dict(cls, typed_dict_name)
    
    # 將 TypedDict 附加到類
    setattr(cls, "_immutable_typed_dict", typed_dict)
    
    # 添加轉換方法
    def to_immutable(self) -> Mapping[str, 'typed_dict']:  # type: ignore # 使用生成的 TypedDict
        return pydantic_to_immutable_map(self)
    
    # 將方法附加到類
    setattr(cls, "to_immutable", to_immutable)
    
    return cls

# 範例：定義 Pydantic 模型
@immutable_map_model
class Address(BaseModel):
    city: str
    zipcode: str

@immutable_map_model
class Employee(BaseModel):
    name: str
    age: int
    address: Address

# 主程式
if __name__ == "__main__":
    # 創建 Pydantic 模型實例
    employee = Employee(
        name="Alice",
        age=30,
        address=Address(city="台北", zipcode="100")
    )

    # 轉換為 immutables.Map
    immutable_employee:Employee = employee.to_immutable()
    immutable_employee.name

    # 輸出結果
    print("immutables.Map 內容:")
    print(immutable_employee)
    print("\n訪問資料:")
    print(f"姓名: {immutable_employee['name']}")
    print(f"年齡: {immutable_employee['age']}")
    print(f"城市: {immutable_employee['address']['city']}")
    print(f"郵遞區號: {immutable_employee['address']['zipcode']}")
    print(f"xxxxx: {immutable_employee.get('name')}")
    immutable_employee['']

    # 驗證不可變性
    try:
        
        immutable_employee["name"] = "Bob"
    except TypeError as e:
        print("\n驗證不可變性:", str(e))

    # 驗證型態註解
    typed_dict = Employee._immutable_typed_dict
    print("\n生成的 TypedDict 型態:")
    print(typed_dict.__annotations__)

immutables.Map 內容:
immutables.Map({'address': immutables.Map({'zipcode': '100', 'city': '台北'}), 'age': 30, 'name': 'Alice'})

訪問資料:
姓名: Alice
年齡: 30
城市: 台北
郵遞區號: 100
xxxxx: Alice

驗證不可變性: 'immutables._map.Map' object does not support item assignment

生成的 TypedDict 型態:
{'name': <class 'str'>, 'age': <class 'int'>, 'address': <class '__main__.EmployeeDict_AddressDict'>}


In [71]:
from typing import Any, Callable, Dict, List, Type, TypeVar
from pydantic import BaseModel
from immutables import Map

# --- 1. 轉換輔助：Pydantic 或 dict/list → immutables.Map / 原生結構 ---
def convert_to_immutable(obj: Any) -> Any:
    if isinstance(obj, BaseModel):
        return Map({k: convert_to_immutable(v) for k, v in obj.dict().items()})
    elif isinstance(obj, dict):
        return Map({k: convert_to_immutable(v) for k, v in obj.items()})
    elif isinstance(obj, list):
        return [convert_to_immutable(v) for v in obj]
    else:
        return obj

# --- 2. 轉回輔助：Map / list → 原生 dict/list，再由 Pydantic 解析 ---
T = TypeVar("T", bound=BaseModel)
def map_to_py(obj: Any) -> Any:
    if isinstance(obj, Map):
        return {k: map_to_py(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [map_to_py(v) for v in obj]
    else:
        return obj

def map_to_model(m: Map, model_cls: Type[T]) -> T:
    data = map_to_py(m)
    return model_cls(**data)

# --- 3. create_reducer：只在第一次 state is None 時，把 default_state 轉成 immutable ---
State = Any
Payload = Any
ActionFn = Callable[[State, Payload], State]

def create_reducer(
    default_state: Any,
    *on_clauses: tuple[str, ActionFn]
) -> Callable[[State | None, "Action"], State]:

    default_imm = convert_to_immutable(default_state)

    def reducer(state: State | None, action: "Action") -> State:
        # 首次 init
        if state is None:
            return default_imm

        for action_type, handler in on_clauses:
            if action.type == action_type:
                return handler(state, action.payload)

        return state

    return reducer

# --- 4. Action & on helper 都不變 ---
class Action:
    def __init__(self, type: str, payload: Any = None):
        self.type = type
        self.payload = payload

def on(action_type: str, handler: ActionFn):
    return (action_type, handler)

# --- 5. 根 reducer registry，不做任何轉換，只聚合 slice reducers ---
class Store:
    def __init__(self):
        self._state: Map[str, Any] | None = None
        self._root_reducer = None

    def register_root(self,
        initial_state: Any,
        reducers: Dict[str, Callable[[Any, Action], Any]]
    ):
        # 把整個初始狀態一次轉成 Map
        app_default = convert_to_immutable(initial_state)

        def root_reducer(state: Map[str, Any] | None, action: Action) -> Map[str, Any]:
            current = state or app_default
            for key, slice_reducer in reducers.items():
                slice_state = current[key]  # 一定存在
                updated = slice_reducer(slice_state, action)
                current = current.set(key, updated)
            return current

        self._root_reducer = root_reducer
        self._state = root_reducer(None, Action("INIT"))

    def dispatch(self, action: Action):
        if not self._root_reducer:
            raise ValueError("還沒 register_root")
        self._state = self._root_reducer(self._state, action)

    def get_state(self) -> Map[str, Any]:
        if self._state is None:
            raise ValueError("尚未初始化")
        return self._state

# --- 6. Pydantic models & TypedDict for initial_state ---
from typing_extensions import TypedDict

class AddressModel(BaseModel):
    city: str = "taiwan"
    zipcode: str

class EmployeeModel(BaseModel):
    name: str
    age: int
    skills: List[str]
    address: AddressModel

class DepartmentModel(BaseModel):
    name: str
    employees: List[EmployeeModel]

class CompanyModel(BaseModel):
    name: str
    departments: List[DepartmentModel]

class Address(TypedDict):
    city: str
    zipcode: str

class Employee(TypedDict):
    name: str
    age: int
    skills: List[str]
    address: Address

class Department(TypedDict):
    name: str
    employees: List[Employee]

class Company(TypedDict):
    name: str
    departments: List[Department]

class AppState(TypedDict):
    employees: List[Employee]
    departments: List[Department]
    company: Company

# --- 7. handlers：假設所有傳入都已經是 Map 或原生 python 結構，不再做任何轉換 ---
from immutables import Map as ImmMap

def update_employee_age_handler(state: List[Employee], payload: dict) -> List[Employee]:
    for idx, emp in enumerate(state):
        if emp["name"] == payload["employee_name"]:
            # 改用 emp 而不是 state
            print(f"update_employee_age_handler: name: {emp['name']}  age: {emp.get('age')} -> {payload['age']}")
            return state[:idx] + [emp.set("age", payload["age"])] + state[idx+1:]
    return state

def add_department_handler(state: List[Department], payload: ImmMap) -> List[Department]:
    return state + [payload]

def update_company_name_handler(state: Company, payload: dict) -> Company:
    print(f"update_company_name_handler: name: {state['name']} -> {payload['name']}")
    return state.set("name", payload["name"])

# --- 8. 使用示範 ---
if __name__ == "__main__":
    # 一次性定義初始狀態
    emp0 = EmployeeModel(name="Alice", age=30, skills=["Python","SQL"], address=AddressModel(zipcode="100"))
    dept0 = DepartmentModel(name="Engineering", employees=[emp0])
    comp0 = CompanyModel(name="Tech Corp", departments=[dept0])

    initial_state: AppState = {
        "employees": [emp0.model_dump()],
        "departments": [dept0.model_dump()],
        "company":    comp0.model_dump()
    }

    # 建立 slice reducers
    emp_reducer  = create_reducer(initial_state["employees"], on("UPDATE_EMPLOYEE_AGE", update_employee_age_handler))
    dept_reducer = create_reducer(initial_state["departments"], on("ADD_DEPARTMENT",    add_department_handler))
    comp_reducer = create_reducer(initial_state["company"],    on("UPDATE_COMPANY_NAME", update_company_name_handler))

    # 註冊 root
    store = Store()
    store.register_root(initial_state, {
        "employees":  emp_reducer,
        "departments": dept_reducer,
        "company":     comp_reducer
    })

    # Dispatch 時直接傳 dict payload
    store.dispatch(Action("ADD_DEPARTMENT", convert_to_immutable(DepartmentModel(name="Marketing", employees=[])).unwrap() if False else convert_to_immutable(DepartmentModel(name="Marketing", employees=[]))))
    store.dispatch(Action("UPDATE_EMPLOYEE_AGE", {"employee_name": "Alice", "age": 31}))
    store.dispatch(Action("UPDATE_COMPANY_NAME", {"name": "New Tech Corp"}))

    final = store.get_state()
    print("最終狀態:", final)

    # 轉回 Pydantic
    company_model = map_to_model(final["company"], CompanyModel)
    print(company_model.model_dump_json(indent=2))


update_employee_age_handler: name: Alice  age: 30 -> 31
update_company_name_handler: name: Tech Corp -> New Tech Corp
最終狀態: immutables.Map({'company': immutables.Map({'name': 'New Tech Corp', 'departments': [immutables.Map({'name': 'Engineering', 'employees': [immutables.Map({'skills': ['Python', 'SQL'], 'address': immutables.Map({'zipcode': '100', 'city': 'taiwan'}), 'age': 30, 'name': 'Alice'})]})]}), 'departments': [immutables.Map({'name': 'Engineering', 'employees': [immutables.Map({'skills': ['Python', 'SQL'], 'address': immutables.Map({'zipcode': '100', 'city': 'taiwan'}), 'age': 30, 'name': 'Alice'})]}), immutables.Map({'name': 'Marketing', 'employees': []})], 'employees': [immutables.Map({'skills': ['Python', 'SQL'], 'address': immutables.Map({'zipcode': '100', 'city': 'taiwan'}), 'age': 31, 'name': 'Alice'})]})
{
  "name": "New Tech Corp",
  "departments": [
    {
      "name": "Engineering",
      "employees": [
        {
          "name": "Alice",
          "age": 30,
      

In [61]:
from typing import List, Callable, Any, TypeVar, Tuple
from typing_extensions import TypedDict
from pydantic import BaseModel
from immutables import Map
from functools import reduce

# 定義 Pydantic 模型（用於初始狀態）
class AddressModel(BaseModel):
    city: str = "taiwan"
    zipcode: str

class EmployeeModel(BaseModel):
    name: str
    age: int
    skills: List[str]
    address: AddressModel

class DepartmentModel(BaseModel):
    name: str
    employees: List[EmployeeModel]

class CompanyModel(BaseModel):
    name: str
    departments: List[DepartmentModel]

# 定義 TypedDict（用於型態檢查和 immutables.Map）
class Address(TypedDict):
    city: str
    zipcode: str

class Employee(TypedDict):
    name: str
    age: int
    skills: List[str]
    address: Address

class Department(TypedDict):
    name: str
    employees: List[Employee]

class Company(TypedDict):
    name: str
    departments: List[Department]

class AppState(TypedDict):
    employees: List[Employee]
    departments: List[Department]
    company: Company

# Action 類型（模擬 NgRx 的 action）
class Action:
    def __init__(self, type: str, payload: Any = None):
        self.type = type
        self.payload = payload

# 定義 on 結構
T = TypeVar("T")

def on(action_type: str | type, handler: Callable[[Map[str, Any], Any], Map[str, Any]]) -> Tuple[str | type, Callable[[Map[str, Any], Any], Map[str, Any]]]:
    """定義 action 和 handler 的映射"""
    return (action_type, handler)

# 將 Pydantic 模型轉換為 immutables.Map
def convert_to_immutable(model: BaseModel) -> Map[str, Any]:
    """將 Pydantic 模型轉換為 immutables.Map"""
    def convert_to_dict(obj: Any) -> Any:
        if isinstance(obj, BaseModel):
            return Map({k: convert_to_dict(v) for k, v in obj.dict().items()})
        elif isinstance(obj, list):
            return [convert_to_dict(item) for item in obj]
        return obj
    return convert_to_dict(model)

# 創建 reducer
def create_reducer(initial_state: BaseModel | None, *on_clauses: Tuple[str | type, Callable[[Map[str, Any], Any], Map[str, Any]]]) -> Callable[[Map[str, Any] | None, Action], Map[str, Any]]:
    """創建 reducer，負責初始化和處理 action"""
    initial_map = convert_to_immutable(initial_state) if initial_state else Map()

    def reducer(state: Map[str, Any] | None, action: Action) -> Map[str, Any]:
        # 如果 state 為 None，使用初始狀態
        if state is None:
            return initial_map

        # 查找匹配的 handler
        for action_type, handler in on_clauses:
            if action.type == action_type or action_type == action.__class__:
                return handler(state, action.payload)
        return state

    return reducer

# 定義 handler
def update_employee_age_handler(state: Map[str, Any], payload: Any) -> Map[str, Any]:
    """處理更新員工年齡"""
    if "employee_name" not in payload or "age" not in payload:
        raise ValueError("UPDATE_EMPLOYEE_AGE payload 缺少 'employee_name' 或 'age'")
    employees = state["employees"]
    for i, emp in enumerate(employees):
        if emp["name"] == payload["employee_name"]:
            new_employee = emp.set("age", payload["age"])
            return state.set("employees", employees[:i] + [new_employee] + employees[i+1:])
    return state

def add_department_handler(state: Map[str, Any], payload: Any) -> Map[str, Any]:
    """處理添加部門"""
    if "department" not in payload:
        raise ValueError("ADD_DEPARTMENT payload 缺少 'department'")
    department = convert_to_immutable(payload["department"])
    return state.set("departments", state["departments"] + [department])

def update_company_name_handler(state: Map[str, Any], payload: Any) -> Map[str, Any]:
    """處理更新公司名稱"""
    if "name" not in payload:
        raise ValueError("UPDATE_COMPANY_NAME payload 缺少 'name'")
    company = state["company"]
    new_company = company.set("name", payload["name"])
    return state.set("company", new_company)

# Store 類（狀態管理核心）
class Store:
    def __init__(self):
        self._state: Map[str, Any] | None = None
        self._reducer: Callable[[Map[str, Any] | None, Action], Map[str, Any]] | None = None

    def add_reducer(self, reducer: Callable[[Map[str, Any] | None, Action], Map[str, Any]]):
        """註冊 reducer"""
        self._reducer = reducer
        # 初始化狀態
        self._state = reducer(None, Action("INIT"))
    

    def dispatch(self, action: Action) -> None:
        """分發 action，更新狀態"""
        if self._reducer is None:
            raise ValueError("未註冊 reducer")
        self._state = self._reducer(self._state, action)

    def get_state(self) -> Map[str, Any]:
        """獲取當前狀態"""
        if self._state is None:
            raise ValueError("狀態尚未初始化")
        return self._state

# 使用範例
if __name__ == "__main__":
    # 使用者定義初始狀態（Pydantic）
    initial_employee = EmployeeModel(
        name="Alice",
        age=30,
        skills=["Python", "SQL"],
        address=AddressModel(zipcode="100")
    )

    initial_department = DepartmentModel(
        name="Engineering",
        employees=[initial_employee]
    )

    initial_company = CompanyModel(
        name="Tech Corp",
        departments=[initial_department]
    )

    initial_state = AppState(
        employees=[initial_employee.dict()],
        departments=[initial_department.dict()],
        company=initial_company.dict()
    )

    # 創建 reducer
    app_reducer = create_reducer(
        None,  # 初始狀態在 combine_reducer 中處理
        on("UPDATE_EMPLOYEE_AGE", update_employee_age_handler),
        on("ADD_DEPARTMENT", add_department_handler),
        on("UPDATE_COMPANY_NAME", update_company_name_handler)
    )

    # 組合初始狀態的 reducer
    def combine_initial_state() -> Map[str, Any]:
        return Map({
            "employees": [convert_to_immutable(initial_employee)],
            "departments": [convert_to_immutable(initial_department)],
            "company": convert_to_immutable(initial_company)
        })

    # 初始化 Store
    store = Store()

    # 註冊 reducer
    store.add_reducer(app_reducer)

    # 設置初始狀態
    store._state = combine_initial_state()

    # 輸出初始狀態
    print("初始狀態:", store.get_state())

    # 分發 action 更新員工年齡
    store.dispatch(Action("UPDATE_EMPLOYEE_AGE", {"employee_name": "Alice", "age": 31}))

    # 分發 action 添加部門
    new_department = DepartmentModel(name="Marketing", employees=[])
    store.dispatch(Action("ADD_DEPARTMENT", {"department": new_department}))

    # 分發 action 更新公司名稱
    store.dispatch(Action("UPDATE_COMPANY_NAME", {"name": "New Tech Corp"}))

    # 輸出更新後的狀態
    print("更新後狀態:", store.get_state())

    # 驗證型態（模擬靜態檢查）
    state: AppState = store.get_state()  # mypy 會檢查是否符合 AppState 型態
    print("員工年齡:", state["employees"][0]["age"])
    print("部門數量:", len(state["departments"]))
    print("公司名稱:", state["company"]["name"])

初始狀態: immutables.Map({'company': immutables.Map({'name': 'Tech Corp', 'departments': [{'name': 'Engineering', 'employees': [{'name': 'Alice', 'age': 30, 'skills': ['Python', 'SQL'], 'address': {'city': 'taiwan', 'zipcode': '100'}}]}]}), 'departments': [immutables.Map({'name': 'Engineering', 'employees': [{'name': 'Alice', 'age': 30, 'skills': ['Python', 'SQL'], 'address': {'city': 'taiwan', 'zipcode': '100'}}]})], 'employees': [immutables.Map({'skills': ['Python', 'SQL'], 'address': {'city': 'taiwan', 'zipcode': '100'}, 'age': 30, 'name': 'Alice'})]})
更新後狀態: immutables.Map({'company': immutables.Map({'name': 'New Tech Corp', 'departments': [{'name': 'Engineering', 'employees': [{'name': 'Alice', 'age': 30, 'skills': ['Python', 'SQL'], 'address': {'city': 'taiwan', 'zipcode': '100'}}]}]}), 'departments': [immutables.Map({'name': 'Engineering', 'employees': [{'name': 'Alice', 'age': 30, 'skills': ['Python', 'SQL'], 'address': {'city': 'taiwan', 'zipcode': '100'}}]}), immutables.Map({'nam

In [ ]:
from immutables import Map
from pydantic import BaseModel
from typing import Mapping, Any
from typing_extensions import TypedDict  # 支援結構化字典型態

# 定義 Pydantic 模型
class Address(BaseModel):
    city: str
    zipcode: str

class Employee(BaseModel):
    name: str
    age: int
    address: Address

# 定義對應的 TypedDict 型態，用於 immutables.Map
class AddressDict(TypedDict):
    city: str
    zipcode: str

class EmployeeDict(TypedDict):
    name: str
    age: int
    address: AddressDict

# 將 Pydantic 模型轉換為 immutables.Map
def pydantic_to_immutable_map(model: BaseModel) -> Mapping[str, Any]:
    # 將 Pydantic 模型轉為 dict
    data = model.dict()
    
    # 遞迴轉換嵌套結構為 immutables.Map
    def convert_to_immutable(obj: Any) -> Any:
        if isinstance(obj, dict):
            return Map({k: convert_to_immutable(v) for k, v in obj.items()})
        elif isinstance(obj, list):
            return [convert_to_immutable(item) for item in obj]
        else:
            return obj
    
    return convert_to_immutable(data)

# 主程式
if __name__ == "__main__":
    # 創建 Pydantic 模型實例
    employee = Employee(
        name="Alice",
        age=30,
        address=Address(city="台北", zipcode="100")
    )

    # 轉換為 immutables.Map
    immutable_employee: Mapping[str, Any] = pydantic_to_immutable_map(employee)
    # 更具體的型態註解（使用 EmployeeDict）
    immutable_employee_typed: Mapping[str, EmployeeDict] = pydantic_to_immutable_map(employee)
    # 輸出結果
    print("immutables.Map 內容:")
    print(immutable_employee)
    print("\n訪問資料:")
    # print(f"姓名: {immutable_employee.get('name')}")
    print(f"姓名: {immutable_employee['name']}")
    print(f"年齡: {immutable_employee['age']}")
    print(f"城市: {immutable_employee['address']['city']}")
    print(f"郵遞區號: {immutable_employee['address']['zipcode']}")

    # 驗證不可變性
    try:
        immutable_employee["name"] = "Bob"  # 應引發錯誤
    except TypeError as e:
        print("\n驗證不可變性:", str(e))

    # 使用型態註解進行訪問
    print("\n使用型態註解訪問:")
    print(f"姓名: {immutable_employee_typed['name']}")
    print(f"城市: {immutable_employee['address']['city']}")
    # print(f"城市: {immutable_employee_typed.get('address').get('city')}")
    # print(f"郵遞區號: {immutable_employee_typed.get('address')['zipcode']}")
    # print(f"郵遞區號: {immutable_employee_typed['name'] ['zipcode']}")

immutables.Map 內容:
immutables.Map({'address': immutables.Map({'zipcode': '100', 'city': '台北'}), 'age': 30, 'name': 'Alice'})

訪問資料:
姓名: Alice
年齡: 30
城市: 台北
郵遞區號: 100

驗證不可變性: 'immutables._map.Map' object does not support item assignment

使用型態註解訪問:
姓名: Alice
城市: 台北


In [43]:
from immutables import Map
from typing import List, Mapping
from pydantic import BaseModel

# 定義嵌套結構的型態
class Address( BaseModel):
    city: str="taiwan"
    zipcode: str

class Employee(BaseModel):
    name: str
    age: int
    skills: List[str]
    address: Address

class Department(BaseModel):
    employees: List[Employee]

# 定義部門的型態，部門是一個 Map，鍵是部門名稱，值是員工清單
DepartmentMap = Mapping[str, List[Employee]]
CompanyMap = Mapping[str, DepartmentMap]

# 創建員工資料
def create_company_data() -> CompanyMap:
    
    address1:Address = Map({"city": "台北", "zipcode": "100"})
    address2:Address = Map({"city": "高雄", "zipcode": "800"})
    address3:Address = Map({"city": "台中", "zipcode": "400"})

    # 員工資料
    employee1:Employee = Map({
        "name": "Alice",
        "age": 30,
        "skills": ["Python", "SQL"],
        "address": address1
    })
    employee1["address"]["zipcode"]
    employee2:Employee = Map({
        "name": "Bob",
        "age": 25,
        "skills": ["Java", "Docker"],
        "address": address2
    })
    employee3:Employee = Map({
        "name": "Charlie",
        "age": 35,
        "skills": ["C++", "Kubernetes"],
        "address": address3
    })

    # 部門資料
    engineering_dept:Department = Map({
        "employees": [employee1, employee2]
    })
    marketing_dept:Department = Map({
        "employees": [employee3]
    })
    marketing_dept["employees"]
    # 公司資料
    company_data:CompanyMap = Map({
        "Engineering": engineering_dept,
        "Marketing": marketing_dept
    })

    return company_data

# 訪問和操作資料
def print_company_data(company: CompanyMap) -> None:
    for dept_name, dept_data in company.items():
        print(f"部門: {dept_name}")
        employees = dept_data["employees"]
        for emp in employees:
            print(f"  員工: {emp['name']}")
            print(f"    年齡: {emp['age']}")
            print(f"    技能: {', '.join(emp['skills'])}")
            print(f"    地址: {emp['address']['city']} ({emp['address']['zipcode']})")

# 範例：衍生新的 Map（模擬更新資料）
def update_employee_age(company: CompanyMap, dept: str, emp_name: str, new_age: int) -> CompanyMap:
    dept_data = company[dept]
    employees = dept_data["employees"]
    for i, emp in enumerate(employees):
        if emp["name"] == emp_name:
            # 創建新的員工資料
            new_emp = Employee(
                name=emp.name,
                age=new_age,
                skills=emp.skills,
                address=emp.address
            )
            # 創建新的員工清單
            new_employees = employees[:i] + [new_emp] + employees[i+1:]
            # 創建新的部門資料
            new_dept = dept_data.set("employees", new_employees)
            # 創建新的公司資料
            return company.set(dept, new_dept)
    return company

# 主程式
if __name__ == "__main__":
    # 創建公司資料
    company = create_company_data()

    # 輸出公司資料
    print("原始公司資料:")
    print_company_data(company)

    # 更新員工 Alice 的年齡
    updated_company = update_employee_age(company, "Engineering", "Alice", 31)

    # 輸出更新後的資料
    print("\n更新後的公司資料:")
    print_company_data(updated_company)

    # 驗證原始資料未被修改
    print("\n驗證原始資料未變:")
    print(f"Alice 的原始年齡: {company['Engineering']['employees'][0].age}")  # 應為 30

原始公司資料:
部門: Marketing
  員工: Charlie
    年齡: 35
    技能: C++, Kubernetes
    地址: 台中 (400)
部門: Engineering
  員工: Alice
    年齡: 30
    技能: Python, SQL
    地址: 台北 (100)
  員工: Bob
    年齡: 25
    技能: Java, Docker
    地址: 高雄 (800)


AttributeError: 'immutables._map.Map' object has no attribute 'name'

In [49]:
from immutables import Map
from typing import List, Mapping, Union
from typing_extensions import TypedDict  # 支援結構化字典型態（Python 3.8+）

# 定義嵌套結構的型態
class Address(TypedDict):
    city: str 
    zipcode: str

class Employee(TypedDict):
    name: str
    age: int
    skills: List[str]
    address: Address

class Department(TypedDict):
    employees: List[Employee]
    
# 定義部門的型態，部門是一個 Map，鍵是部門名稱，值是員工清單
DepartmenntMap = Mapping[str, List[Employee]]
CompanyMap = Mapping[str, DepartmentMap]

# 創建員工資料
def create_company_data() -> CompanyMap:
    # 地址資料
    address1:Address = Map({"city": "台北", "zipcode": "100"})
    address2:Address = Map({"city": "高雄", "zipcode": "800"})
    address3:Address = Map({"city": "台中", "zipcode": "400"})

    # 員工資料
    employee1:Employee = Map({
        "name": "Alice",
        "age": 30,
        "skills": ["Python", "SQL"],
        "address": address1
    })
    print(employee1["address"]["city"])
    employee1["address"]["zipcode"]
    employee2:Employee = Map({
        "name": "Bob",
        "age": 25,
        "skills": ["Java", "Docker"],
        "address": address2
    })
    employee3:Employee = Map({
        "name": "Charlie",
        "age": 35,
        "skills": ["C++", "Kubernetes"],
        "address": address3
    })

    # 部門資料
    engineering_dept:Department = Map({
        "employees": [employee1, employee2]
    })
    marketing_dept:Department = Map({
        "employees": [employee3]
    })
    marketing_dept["employees"]
    # 公司資料
    company_data:CompanyMap = Map({
        "Engineering": engineering_dept,
        "Marketing": marketing_dept
    })
    return company_data

# 訪問和操作資料
def print_company_data(company: CompanyMap) -> None:
    for dept_name, dept_data in company.items():
        print(f"部門: {dept_name}")
        employees = dept_data["employees"]
        for emp in employees:
            print(f"  員工: {emp['name']}")
            print(f"    年齡: {emp['age']}")
            print(f"    技能: {', '.join(emp['skills'])}")
            print(f"    地址: {emp['address']['city']} ({emp['address']['zipcode']})")

# 範例：衍生新的 Map（模擬更新資料）
def update_employee_age(company: CompanyMap, dept: str, emp_name: str, new_age: int) -> CompanyMap:
    dept_data = company[dept]
    employees = dept_data["employees"]
    for i, emp in enumerate(employees):
        if emp["name"] == emp_name:
            
            # 創建新的員工資料
            new_emp:Employee = emp.set("age", new_age)
            
            # 創建新的員工清單
            new_employees:Employee = employees[:i] + [new_emp] + employees[i+1:]
            
            # 創建新的部門資料
            new_dept:Department = dept_data.set("employees", new_employees)
            # 創建新的公司資料
            return company.set(dept, new_dept)
    return company

# 主程式
if __name__ == "__main__":
    # 創建公司資料
    company = create_company_data()

    # 輸出公司資料
    print("原始公司資料:")
    print_company_data(company)

    # 更新員工 Alice 的年齡
    updated_company = update_employee_age(company, "Engineering", "Alice", 31)

    # 輸出更新後的資料
    print("\n更新後的公司資料:")
    print_company_data(updated_company)

    # 驗證原始資料未被修改
    print("\n驗證原始資料未變:")
    print(f"Alice 的原始年齡: {company['Engineering']['employees'][0]['age']}")  # 應為 30

台北
原始公司資料:
部門: Marketing
  員工: Charlie
    年齡: 35
    技能: C++, Kubernetes
    地址: 台中 (400)
部門: Engineering
  員工: Alice
    年齡: 30
    技能: Python, SQL
    地址: 台北 (100)
  員工: Bob
    年齡: 25
    技能: Java, Docker
    地址: 高雄 (800)

更新後的公司資料:
部門: Marketing
  員工: Charlie
    年齡: 35
    技能: C++, Kubernetes
    地址: 台中 (400)
部門: Engineering
  員工: Alice
    年齡: 31
    技能: Python, SQL
    地址: 台北 (100)
  員工: Bob
    年齡: 25
    技能: Java, Docker
    地址: 高雄 (800)

驗證原始資料未變:
Alice 的原始年齡: 30


In [1]:
# 不使用 __slots__ 的情況  
import sys  
from typing import NamedTuple, Optional  
  
class ActionWithoutSlots(NamedTuple):  
    type: str  
    payload: Optional[object] = None  
  
# 使用 __slots__ 的情況  
class ActionWithSlots:  
    __slots__ = ('type', 'payload')  
      
    def __init__(self, type: str, payload: Optional[object] = None):  
        self.type = type  
        self.payload = payload  
  
# 創建實例  
action1 = ActionWithoutSlots("increment")  
action2 = ActionWithSlots("increment")  
  
# 計算記憶體大小  
size1 = sys.getsizeof(action1)  
size2 = sys.getsizeof(action2)  
  
print(f"不使用 __slots__: {size1} bytes")  
print(f"使用 __slots__: {size2} bytes")  
print(f"記憶體節省: {size1 - size2} bytes ({(1 - size2/size1) * 100:.2f}%)")

不使用 __slots__: 56 bytes
使用 __slots__: 48 bytes
記憶體節省: 8 bytes (14.29%)


In [4]:
!pip install pystorex==0.1.7

  Attempting uninstall: pystorex
    Found existing installation: pystorex 0.1.6
    Uninstalling pystorex-0.1.6:
      Successfully uninstalled pystorex-0.1.6


In [ ]:
from pystorex import (
    create_store,
    create_reducer,
    on,
    create_effect,
    create_selector,
)
from pydantic import BaseModel
from pystorex.actions import create_action

# 1. 定义你的 State 模型
class CounterState(BaseModel):
    count: int = 0

# 2. 创建 Actions
increment = create_action("increment")
decrement = create_action("decrement")
reset = create_action("reset", lambda value: value)
increment_by = create_action("incrementBy", lambda amount: amount)

# 3. 创建 Reducer
def counter_handler(state: CounterState, action):
    new_state = state.model_copy()
    if action.type == increment.type:
        new_state.count += 1
    elif action.type == decrement.type:
        new_state.count -= 1
    return new_state

counter_reducer = create_reducer(
    CounterState(),
    on(increment, counter_handler),
    on(decrement, counter_handler),
)

# 4. 初始化 Store（不再传初始 state）
store = create_store()

#    register_root 内部会发出 `[Root] Init Store`，
#    并用每个 reducer.initial_state 填充 store.state
store.register_root({"counter": counter_reducer})

# 5. 订阅 state 变化
#    由于 register_root 时我们对 select() 做了 skip(1)，
#    这里不会收到那次空的初始注入。
store.select(lambda s: s["counter"].count).subscribe(
    lambda new: print("Count:", new[1])
)

# 6. Dispatch Actions
store.dispatch(increment())  # Count: 1
store.dispatch(increment())  # Count: 2
store.dispatch(decrement())  # Count: 1

Count: 1
Count: 2
Count: 1


Action(type='decrement', payload=None)

In [ ]:
import time
from typing import Optional
from pydantic import BaseModel
from reactivex import operators as ops

from pystorex.actions import create_action
from pystorex import create_store, create_reducer, on, create_effect
from pystorex.store_selectors import create_selector
from pystorex.middleware import LoggerMiddleware

# 1. 定義狀態模型
class CounterState(BaseModel):
    count: int = 0
    loading: bool = False
    error: Optional[str] = None
    last_updated: Optional[float] = None

# 2. 定義 Actions
increment = create_action("increment")
decrement = create_action("decrement")
reset = create_action("reset", lambda value: value)
increment_by = create_action("incrementBy", lambda amount: amount)

load_count_request = create_action("loadCountRequest")
load_count_success = create_action("loadCountSuccess", lambda value: value)
load_count_failure = create_action("loadCountFailure", lambda error: error)

# 3. 定義 Reducer
def counter_handler(state: CounterState, action) -> CounterState:
    new_state = state.copy(deep=True)
    now = time.time()

    if action.type == increment.type:
        new_state.count += 1
        new_state.last_updated = now
    elif action.type == decrement.type:
        new_state.count -= 1
        new_state.last_updated = now
    elif action.type == reset.type:
        new_state.count = action.payload
        new_state.last_updated = now
    elif action.type == increment_by.type:
        new_state.count += action.payload
        new_state.last_updated = now
    elif action.type == load_count_request.type:
        new_state.loading = True
        new_state.error = None
    elif action.type == load_count_success.type:
        new_state.loading = False
        new_state.count = action.payload
        new_state.last_updated = now
    elif action.type == load_count_failure.type:
        new_state.loading = False
        new_state.error = action.payload

    return new_state

counter_reducer = create_reducer(
    CounterState(),
    on(increment, counter_handler),
    on(decrement, counter_handler),
    on(reset, counter_handler),
    on(increment_by, counter_handler),
    on(load_count_request, counter_handler),
    on(load_count_success, counter_handler),
    on(load_count_failure, counter_handler),
)

# 4. 定義 Effects
class CounterEffects:
    @create_effect
    def load_count(self, action_stream):
        return action_stream.pipe(
            ops.filter(lambda action: action.type == load_count_request.type),
            ops.do_action(lambda _: print("Effect: Loading counter...")),
            ops.delay(1.0),
            ops.map(lambda _: load_count_success(42))
        )


# 5. 建立 Store、註冊模組
store = create_store()
store.apply_middleware(LoggerMiddleware)
store.register_root({"counter": counter_reducer})
store.register_effects(CounterEffects)

# 6. 訂閱狀態與測試
get_counter_state = lambda state: state["counter"]
get_count = create_selector(
    get_counter_state,
    result_fn=lambda counter: counter.count or 0
)
store.select(get_count).subscribe(
    lambda c: print(f"Count: {c[1]}")
)

# 7. 執行操作示例
if __name__ == "__main__":
    store.dispatch(increment())
    store.dispatch(increment_by(5))
    store.dispatch(decrement())
    store.dispatch(reset(10))
    store.dispatch(load_count_request())
    # 給 Effects 一些時間
    time.sleep(2)


▶️ dispatching increment
🔄 state before increment: {'counter': CounterState(count=0, loading=False, error=None, last_updated=None)}
Count: 1
✅ state after increment: {'counter': CounterState(count=1, loading=False, error=None, last_updated=1745050111.1340942)}
▶️ dispatching incrementBy
🔄 state before incrementBy: {'counter': CounterState(count=1, loading=False, error=None, last_updated=1745050111.1340942)}
Count: 6
✅ state after incrementBy: {'counter': CounterState(count=6, loading=False, error=None, last_updated=1745050111.1340942)}
▶️ dispatching decrement
🔄 state before decrement: {'counter': CounterState(count=6, loading=False, error=None, last_updated=1745050111.1340942)}
Count: 5
✅ state after decrement: {'counter': CounterState(count=5, loading=False, error=None, last_updated=1745050111.1340942)}
▶️ dispatching reset
🔄 state before reset: {'counter': CounterState(count=5, loading=False, error=None, last_updated=1745050111.1340942)}
Count: 10
✅ state after reset: {'counter': Co

: 